<a href="https://colab.research.google.com/github/AmitAaranya/ASL-Recognition-with-Deep-Learning-PyTorch/blob/main/google-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone GitHub repo
**URL:** https://github.com/AmitAaranya/ASL-Recognition-with-Deep-Learning-PyTorch

In [2]:
!git clone https://github.com/AmitAaranya/ASL-Recognition-with-Deep-Learning-PyTorch.git ./code/

fatal: destination path './code' already exists and is not an empty directory.


In [11]:
!git --git-dir=./code/.git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 966 bytes | 966.00 KiB/s, done.
From https://github.com/AmitAaranya/ASL-Recognition-with-Deep-Learning-PyTorch
   43ef39f..606d3a7  main       -> origin/main
Updating 43ef39f..606d3a7
Fast-forward
 requirements.txt | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [4]:
## Add the folder to system path, so we can import file directly
import sys
sys.path.append('/content/code')

In [13]:
!pip install -r code/requirements.txt > /dev/null

# Load the ASL Image Data
**URL:** https://www.kaggle.com/datasets/lexset/synthetic-asl-alphabet


In [6]:
!echo "{\"username\":\"amitaaranya\",\"key\":\"e8845441bb41bfd733339ef7c8f50fa\"}" > /root/.kaggle/kaggle.json

/bin/bash: line 1: /root/.kaggle/kaggle.json: No such file or directory


In [7]:
!kaggle datasets download -d lexset/synthetic-asl-alphabet

Dataset URL: https://www.kaggle.com/datasets/lexset/synthetic-asl-alphabet
License(s): copyright-authors
100% 6.58G/6.58G [05:50<00:00, 22.2MB/s]
100% 6.58G/6.58G [05:50<00:00, 20.1MB/s]


In [8]:
!mkdir datasets

In [9]:
!unzip synthetic-asl-alphabet.zip -d ./datasets/ > /dev/null

# Exploratory Data Analysis(EDA)

In [5]:
# Imports
import torch
import numpy as np

## Load Data

In [6]:
from data_loader import load_datasets

size = 100
x_train, y_train = load_datasets(size = size)
x_test,  y_test = load_datasets(train_or_test_folder="Test_Alphabet",size = size)

print(f"Train dataset count: {len(x_train)} \nTest dataset count: {len(x_test)} ")

Train dataset count: 24300 
Test dataset count: 2700 


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
from data_loader import alphabet_to_num

def one_hot_coding(unique_labels, data):
    y_oh = torch.zeros(len(data),unique_labels)
    for i, class_word in enumerate(data):
        index = alphabet_to_num(class_word)
        y_oh[i][index] = 1

    return y_oh

y_train_oh = one_hot_coding(len(np.unique(y_train)),y_train)
y_test_oh = one_hot_coding(len(np.unique(y_train)),y_test)

In [11]:
x_train.shape,y_train.shape

((24300, 3, 100, 100), (24300,))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=27, kernel_size=5, padding=2)
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=4, stride=4)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=27, out_channels=27, kernel_size=5, padding=2)
        # Another max pooling layer
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=4)
        # Fully connected layer
        self.fc1 = nn.Linear(9 * 6 * 6, 27)  # 3x3 is the size after two max pooling layers

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 9 * 6 * 6)  # Flatten the output of conv2 layer
        x = self.fc1(x)
        return x

# Instantiate the model
model = ConvNet()

# Print the model architecture
print(model)

In [31]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(),lr=0.00001)
# scheduler = StepLR(optimizer, step_size=5, gamma=0.00001)
batch_size = 32

x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_oh_tensor = torch.tensor(y_train_oh, dtype=torch.float32).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test_oh_tensor = torch.tensor(y_test_oh, dtype=torch.float32).to(device)

train_loader = DataLoader(TensorDataset(x_train_tensor,y_train_oh_tensor),
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_tensor,y_test_oh_tensor),
                          batch_size=batch_size)

<ipython-input-31-954ebab3ef2a>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_oh_tensor = torch.tensor(y_train_oh, dtype=torch.float32).to(device)
<ipython-input-31-954ebab3ef2a>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_oh_tensor = torch.tensor(y_test_oh, dtype=torch.float32).to(device)


In [13]:
len(train_loader.dataset),len(test_loader.dataset)

(24300, 2700)

In [14]:
print(train_loader.batch_size)

32


## Define CNN model

In [36]:
def train_model(model, criterion, optimizer, device, num_epochs=100):
    model.to(device)  # Move the model to the device outside the loop
    for epoch in range(num_epochs):
        running_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to device
            # print(f"Inputs size from train_loader: {inputs.size()}")  # Print sizes for debugging
            # print(f"Labels size from train_loader: {labels.size()}")
            optimizer.zero_grad()
            outputs = model(inputs)
            # print(f"Output Tensor size from Model: {outputs.size()}")
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # scheduler.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss}")
        # Learnig rate:{scheduler.get_last_lr()}

# Call the function to train the model
train_model(model=model, criterion=criterion, optimizer=optimizer, device=device)

Epoch [1/100], Train Loss: 5.566555512868412e-07
Epoch [2/100], Train Loss: 5.488407235652008e-07
Epoch [3/100], Train Loss: 5.433463849728404e-07
Epoch [4/100], Train Loss: 5.389950238991651e-07
Epoch [5/100], Train Loss: 5.325587896132929e-07
Epoch [6/100], Train Loss: 5.252590707269096e-07
Epoch [7/100], Train Loss: 5.218594114613676e-07
Epoch [8/100], Train Loss: 5.141722453910369e-07
Epoch [9/100], Train Loss: 5.072699671876699e-07
Epoch [10/100], Train Loss: 5.027321887388138e-07
Epoch [11/100], Train Loss: 4.960997173835858e-07
Epoch [12/100], Train Loss: 4.900264898013675e-07
Epoch [13/100], Train Loss: 4.855230223126568e-07
Epoch [14/100], Train Loss: 4.763003460742202e-07
Epoch [15/100], Train Loss: 4.724248793930434e-07
Epoch [16/100], Train Loss: 4.6734745141433276e-07
Epoch [17/100], Train Loss: 4.636485981542912e-07
Epoch [18/100], Train Loss: 4.5406285536958966e-07
Epoch [19/100], Train Loss: 4.482251163509269e-07
Epoch [20/100], Train Loss: 4.4466357503684155e-07
Epoch 

In [38]:
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs, 1)
          _, labels_indices = torch.max(labels, 1)  # Convert one-hot labels to indices
          total += labels.size(0)
          correct += (predicted == labels_indices).sum().item()
    accuracy = correct / total
    print(f"Total accurate predictions: {correct} out of {total}")
    print(f"Accuracy on test set: {accuracy:.4f}")

test_model(model=model,test_loader=test_loader)

Total accurate predictions: 2539 out of 2700
Accuracy on test set: 0.9404
